In [5]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input, concatenate
from sklearn.model_selection import train_test_split

# 데이터

In [6]:
PATH = 'C:/study/keras/keras_data/stock/'
samsung = pd.read_csv(PATH + 'samsung.csv', index_col = 0) 
print(samsung)

# y = datasets['시가', '종가', '고가', '저가', '거래량']
# samsung = np.array([range(100), range(301, 401)]).T
# amore = np.array([range(101, 201), range(411, 511), range(150, 250)]).T
# y = np.array(range(2001, 2101))

# print(x1.shape)
# print(x2.shape)
# print(y.shape)

# x1_train, x1_test, x2_train, x2_test, y_train, y_test = train_test_split(x1, x2, y, train_size=0.7, random_state=3333) 

# print(x1_train.shape, x1_test.shape, x2_train.shape, x2_test.shape, y_train.shape, y_test.shape)

# x_train = x_train.reshape(-1, 4, 1)
# x_test = x_test.reshape(-1, 4, 1)
# pre_data = pre_data.reshape(-1, 4, 1)

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xc0 in position 0: invalid start byte

In [10]:
# 모델 1
input1 = Input(shape=(2, ))
dense1 = Dense(11, activation='relu', name='ds11')(input1) # summary를 봤을 때 보기쉽게 이름 지정
dense2 = Dense(12, activation='relu', name='ds12')(dense1)
dense3 = Dense(13, activation='relu', name='ds13')(dense2)
output1 = Dense(14, activation='relu', name='ds14')(dense3)

# 모델 2 
input2 = Input(shape=(3, ))
dense21 = Dense(21, activation='linear', name='ds21')(input2)
dense22 = Dense(22, activation='linear', name='ds22')(dense21)
output2 = Dense(23, activation='linear', name='ds23')(dense22)
       
# 병합
merge1 = concatenate([output1, output2], name = 'mg1')
merge2 = Dense(12, activation='relu', name='mg2')(merge1)
merge3 = Dense(13, name='mg3')(merge2) 
last_output = Dense(1, name = 'last')(merge3)

model = Model(inputs=[input1, input2], outputs=[last_output])

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 2)]          0           []                               
                                                                                                  
 ds11 (Dense)                   (None, 11)           33          ['input_1[0][0]']                
                                                                                                  
 input_2 (InputLayer)           [(None, 3)]          0           []                               
                                                                                                  
 ds12 (Dense)                   (None, 12)           144         ['ds11[0][0]']                   
                                                                                              

# 컴파일, 훈련

In [11]:
model.compile(loss='mse', optimizer= 'adam')
model.fit([x1_train, x2_train], y_train, epochs=16, batch_size=3)

Epoch 1/16
24/24 [==============================] - 0s 1ms/step - loss: 2769412.2500
Epoch 2/16
24/24 [==============================] - 0s 890us/step - loss: 337599.7812
Epoch 3/16
24/24 [==============================] - 0s 872us/step - loss: 50055.1406
Epoch 4/16
24/24 [==============================] - 0s 949us/step - loss: 15753.7090
Epoch 5/16
24/24 [==============================] - 0s 938us/step - loss: 15440.9004
Epoch 6/16
24/24 [==============================] - 0s 999us/step - loss: 14902.7139
Epoch 7/16
24/24 [==============================] - 0s 1ms/step - loss: 13862.8066
Epoch 8/16
24/24 [==============================] - 0s 937us/step - loss: 14908.2422
Epoch 9/16
24/24 [==============================] - 0s 995us/step - loss: 14129.9971
Epoch 10/16
24/24 [==============================] - 0s 962us/step - loss: 12969.6250
Epoch 11/16
24/24 [==============================] - 0s 988us/step - loss: 13699.6201
Epoch 12/16
24/24 [==============================] - 0s 1ms/step

# 평가

In [13]:
loss = model.evaluate([x1_test, x2_test], y_test)
print("loss(mse): :", loss)

1/1 [==============================] - 0s 16ms/step - loss: 14430.5723
loss(mse): : 14430.572265625
